In [ ]:
import sys
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from pathlib import Path

load_dotenv(override=True)

OPENAI_API_KEY = os.environ["OPENAI_API_CHATBOT_TEST_KEY_INTERNAL"]
MONGO_URI = os.environ["MONGO_URI"]
EMBEDDING_MODEL_NAME = os.environ["EMBEDDING_MODEL_NAME"]
EMBEDDING_DIMENSIONS = os.environ["EMBEDDING_DIMENSIONS"]
CHAT_MODEL_NAME = os.environ["CHAT_MODEL_NAME"]

DB_NAME = "gaia"
COLLECTION_NAME = "documents"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"
MAX_CHUNKS_TO_RETRIEVE=10
CHUNK_MIN_RELEVANCE_SCORE=0.2

MAX_TOKENS_FOR_RESPONSE = 500
CHAT_MODEL_TEMPERATURE=0.3
CHAT_MODEL_FREQ_PENALTY=0
CHAT_MODEL_PRES_PENALTY=0
MEMORY_WINDOW_SIZE = 5  # Number of turns to keep in memory before summarizing
SHOW_VERBOSE=True


PARENT_PATH = Path.cwd().parent
EVA_SETTINGS_PATH = PARENT_PATH / 'evasettings'
EVA_SETTINGS_ENVIRONMENT_DIRECTORY = 'local'

In [ ]:
import sys
from pathlib import Path

# Get the project root by navigating two levels up from the current notebook directory
project_root = Path.cwd().parent.parent

# Define paths to models and vectordatabases directories relative to the project root
models_path = project_root / 'scripts' / 'models'
vectordatabases_path = project_root / 'scripts' / 'vectordatabases'

# Add the paths to sys.path if they're not already in it
if str(models_path) not in sys.path:
    sys.path.append(str(models_path))
if str(vectordatabases_path) not in sys.path:
    sys.path.append(str(vectordatabases_path))

# Print sys.path for debugging
print(f"Models Path: {models_path}")
print(f"Vector Databases Path: {vectordatabases_path}")
print(sys.path)

# Try importing the modules now
from models import model_rag
from vectordatabases import BaseDB

In [ ]:
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.runnables import RunnableSequence
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.schema import HumanMessage, SystemMessage

class RAG:
    def __init__(self, chat_data):
        self.chat_data = chat_data

    ## Public Methods
    def get_response(self):
        self.llm_eva = ChatOpenAI(
            model_name=self.chat_data.rag_settings.chat_model_name,
            temperature=self.chat_data.rag_settings.temperature,
            max_tokens=self.chat_data.rag_settings.max_tokens_for_response,
            openai_api_key=self.chat_data.llm_settings.llm_key
        )

        self.summarized_history = ""
        self.memory = None
        if self.chat_data.chat_history:
            self.summarized_history, self.memory = self._summarize_history()
        
        # Detect the intent first
        intent_name = self._detect_intent()
        print("Detected Intent: ", intent_name)

        # Now invoke the QA model to get the response
        qa = self._get_qa_instance(intent_name)
        result = qa.invoke({"question": self.chat_data.user_input})

        response_text = result.get("answer", "")
        sources_list = result.get("sources", "")

        # return response_text, sources_list
    
        return model_rag.ChatResponse(response=response_text, sources=sources_list)

    ## Private Methods
    def _load_template(self, project_template_directory_name, template_file_name):
        project_template_directory_path = os.path.join(EVA_SETTINGS_PATH, project_template_directory_name, EVA_SETTINGS_ENVIRONMENT_DIRECTORY)
        template_file_path = project_template_directory_path+ '/' + template_file_name
        with open(template_file_path, "r") as file:
            return file.read()

    def _build_intent_detection_prompt(self):
        # Load intent detection template
        intent_detection_template = self._load_template(
            self.chat_data.prompt_template_directory_name, 
            self.chat_data.intent_detection_prompt_template_file_name
        )
        
        # Dynamically build the intent list from intent details
        intent_list = "\n".join(
            [f'- "{intent_name}": {intent_data.description}' for intent_name, intent_data in self.chat_data.intent_details].items()]
        )
        
        # Build the full prompt for intent detection
        prompt = intent_detection_template.format(
            user_input=self.chat_data.user_input,
            history=self.summarized_history,  # Use summarized history here
            intent_list=intent_list
        )
        return prompt

    def _build_chat_prompt(self, intent_name):
        # Load base template
        base_template = self._load_template(self.chat_data.prompt_template_directory_name, self.chat_data.base_prompt_template_file_name])
        
        # Load intent-specific template or default to generic message
        if intent_name == "none":
            intent_template = ""
        else:
            intent_filename = self.chat_dataintent_details.get(intent_name).filename
            intent_template = self._load_template(self.chat_data.prompt_template_directory_name, intent_filename)
        
        # Build the full prompt using the base and intent templates
        return base_template.format(
            subinstructions=intent_template,
            history=self.summarized_history,  # Pass the summarized history for context
            summaries="{summaries}",
            question="{question}"
        )

    def _get_qa_retriever(self):
        llm_embeddings = OpenAIEmbeddings(
            model=self.chat_data.llm_settings.embedding_model_name,
            openai_api_key=self.chat_data.llm_settings.llm_key
        )
    
        db_instance = BaseDB().get_vector_db(
            self.chat_data.db_type,
            self.chat_data.db_settings,
            llm_embeddings
        )
        vector_store = db_instance.vector_index
    
        qa_retriever = vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": self.chat_data.rag_settings.max_chunks_to_retrieve.value, "score_threshold": self.chat_data.rag_settings.retrieved_chunks_min_relevance_score.value})
        
        return qa_retriever
    
        
    def _summarize_history(self):
        if not self.chat_data.chat_history:
            return "", None
        
        history = ChatMessageHistory()
        for conv in self.chat_data.chat_history:
            history.add_user_message(conv.human)
            history.add_ai_message(conv.ai)
            
        memory = ConversationSummaryMemory.from_messages(
            llm=self.llm_eva,
            chat_memory=history,
            return_messages=True,
            memory_key="history",
            input_key="question"
        )

        # Get the summarized history
        #summarized_history = memory.load_memory_variables({}).get('history', '')
        summarized_history = memory.buffer

        print('summarized_history:', summarized_history)
       
        return summarized_history, memory

    def _get_qa_instance(self, intent_name):
        dynamic_prompt_content = self._build_chat_prompt(intent_name)
            
        prompt_template = PromptTemplate(
            template=dynamic_prompt_content,
            input_variables=['summaries', 'question']
        )
    
        qa_retriever = self._get_qa_retriever()
        
        qa = RetrievalQAWithSourcesChain.from_chain_type(
            llm=self.llm_eva,
            chain_type="stuff",
            retriever=qa_retriever,
            return_source_documents=False,
            chain_type_kwargs={
                "verbose": SHOW_VERBOSE,
                "prompt": prompt_template,
                "memory": self.memory  # Include memory if available
            } if self.memory else {
                "verbose": SHOW_VERBOSE,
                "prompt": prompt_template
            }
        )
    
        return qa

    def _detect_intent(self):
        intent_prompt = self._build_intent_detection_prompt()
        
        prompt_template = PromptTemplate(
            template=intent_prompt,
            input_variables=["user_input", "history", "intent_list"]
        )
        
        intent_chain = RunnableSequence(prompt_template, self.llm_eva)
        intent_result = intent_chain.invoke({
            "user_input": self.chat_data.user_input,  
            "history": self.summarized_history,  
            "intent_list": "\n".join([f'- "{intent_name}"' for intent_name in self.chat_data.intent_details.keys()])
        })

        detected_intent = intent_result.content.strip()
        if detected_intent not in self.chat_data.intent_details:
            return "none"
        return detected_intent




In [ ]:
conversation_history = []


def get_chatbot_response(payload: model_rag.ChatRequest):
    try:
        chat_processor = rag.RAG(payload)
        response = chat_processor.get_response()
        return response
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Chatbot request failed: {str(e)}")


def call_chatbot_endpoint(user_input_text):
    global conversation_history
    
    data = {
        "db_type": "mongodb",
        "db_settings": {
            "uri": MONGO_URI,  
            "db_name": DB_NAME,  
            "collection_name": COLLECTION_NAME,  
            "vector_index_name": ATLAS_VECTOR_SEARCH_INDEX_NAME,  
            "vector_similarity_function": "cosine"  
        },
        "llm_settings": {
            "llm_key": OPENAI_API_KEY,  
            "vector_dimension_size": EMBEDDING_DIMENSIONS,  
            "embedding_model_name": EMBEDDING_MODEL_NAME  
        },
        "rag_settings": {
            "chat_model_name": CHAT_MODEL_NAME,  
            "max_chunks_to_retrieve": MAX_CHUNKS_TO_RETRIEVE,  
            "retrieved_chunks_min_relevance_score": CHUNK_MIN_RELEVANCE_SCORE,
            "max_tokens_for_response": MAX_TOKENS_FOR_RESPONSE,  
            "temperature": CHAT_MODEL_TEMPERATURE,  
            "frequency_penalty": CHAT_MODEL_FREQ_PENALTY,
            "presence_penalty": CHAT_MODEL_PRES_PENALTY
        },
        "user_input": user_input_text,
        "chat_history": conversation_history,
        "prompt_template_directory_name": "gaia",  
        "base_prompt_template_file_name": "base_template.txt", 
        "intent_detection_prompt_template_file_name": "detect_intent.txt", 
        "intent_details": {
            "diagnosis": {
                "filename": "diagnosis.txt",
                "description": "This intent covers diagnosis-related queries for crop issues."
            },
            "symptoms_identification": {
                "filename": "symptoms_identification.txt",
                "description": "This intent helps identify symptoms for a specific pest or problem."
            },
            "pest_list": {
                "filename": "pest_list.txt",
                "description": "This intent provides a list of pests affecting specific crops in a given location."
            },
            "ipm_pest_management": {
                "filename": "ipm_pest_management.txt",
                "description": "This intent provides integrated pest management advice, including biocontrol and chemical recommendations."
            },
            "chemical_handling_safety": {
                "filename": "chemical_handling_safety.txt",
                "description": "This intent provides safety advice regarding the handling and application of chemicals."
            },
            "invasive_pest_status": {
                "filename": "invasive_pest_status.txt",
                "description": "This intent provides the current status of invasive pests in a specific region."
            },
            "dosage_recommendations": {
                "filename": "dosage_recommendations.txt",
                "description": "This intent provides dosage recommendations for specific chemical or biocontrol products."
            }
        }
    }

    # chat_processor = RAG(data)
    # response_text, sources_list = chat_processor.get_response()
    
    response = get_chatbot_response(payload=data)
    
    conversation_history.append({
        "human": user_input_text,  
        "ai": response.response_text  
    })
    
    return response.response_text

In [ ]:
handle_user_input("What is fall army worm?")

In [ ]:
# handle_user_input("Which regions fall army worm affect?")

In [ ]:
# handle_user_input("Hi, what are pests that affect tomatoes in Jamaica?")

In [ ]:
# handle_user_input("And what are other pest affecting that region?")

In [ ]:
# handle_user_input("I didn't mentiond rice but answered for rice. why?")